In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
!pip install datasets

In [79]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from sklearn.datasets import load_files
import matplotlib.image as mpimg
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from datasets import load_dataset
import os, os.path
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import numpy as np
import cv2
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [80]:
train_data_dir = '/content/drive/MyDrive/Thoa/DataAugmentedOrigin/train'
test_data_dir = '/content/drive/MyDrive/Thoa/DataAugmentedOrigin/test'
val_data_dir = '/content/drive/MyDrive/Thoa/DataAugmentedOrigin/val'

labels = ['Cam thao', 'Cay bac ha', 'Cay luu ly', 'Cay mac co', 'Cay ma de', 'Co thom', 'Dinh huong', 'Dong trung ha thao', 'Hat de ngua', 'Hoa dau biec', 'Ich mau', 'Ngai cuu', 'Nha dam', 'Toi', 'Trinh nu hoang cung', 'Y di']

In [81]:
def load_image(data_dir):
    dataset = load_dataset("imagefolder", data_dir = data_dir)
    images = dataset['train']['image']
    labels = dataset['train']['label']
    return images, labels

In [82]:
X_train, y_train = load_image(train_data_dir)
X_test, y_test = load_image(test_data_dir)
X_val, y_val = load_image(val_data_dir)

Resolving data files:   0%|          | 0/3924 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/208 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/208 [00:00<?, ?it/s]

In [83]:
def one_hot_coding(y):
    # Create a one-hot encoder
    encoder = OneHotEncoder()

    # Fit and transform the data
    one_hot_data = encoder.fit_transform(np.array(y).reshape(-1, 1)).toarray()

    return one_hot_data

def decode_one_hot(one_hot_data, labels):
     # Get the index of the non-zero element in each row
    decoded_data_to_int = np.argmax(one_hot_data, axis=1)

    # Assign the corresponding unique values
    decoded_data_to_origin = [labels[i] for i in decoded_data_to_int]

    return decoded_data_to_int, np.array(decoded_data_to_origin)

In [84]:
y_train = one_hot_coding(y_train)
y_test = one_hot_coding(y_test)
y_val = one_hot_coding(y_val)

In [85]:
def convert_image_to_array_form(X):
    image_array = []
    for image in X:
        image = np.array(image)
        new_size = (224, 224)
        img_resized = cv2.resize(image, new_size)
        image_array.append(img_resized)
    return np.array(image_array)

In [86]:
X_train = convert_image_to_array_form(X_train)
X_test = convert_image_to_array_form(X_test)
X_val = convert_image_to_array_form(X_val)

In [87]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3), padding = 'same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.3),

    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.3),


    keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.4),

    keras.layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.4),

    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),

    keras.layers.Dense(256, activation='relu'),  # Giảm kích thước lớp Dense
    keras.layers.Dropout(0.5),

    keras.layers.Dense(128, activation='relu'),  # Giảm kích thước lớp Dense
    keras.layers.Dropout(0.5),

    keras.layers.Dense(16, activation='softmax')
])

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'Recall']
)

In [88]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 224, 224, 32)      896       
                                                                 
 batch_normalization_24 (Ba  (None, 224, 224, 32)      128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_24 (MaxPooli  (None, 112, 112, 32)      0         
 ng2D)                                                           
                                                                 
 dropout_15 (Dropout)        (None, 112, 112, 32)      0         
                                                                 
 conv2d_25 (Conv2D)          (None, 112, 112, 64)      18496     
                                                                 
 batch_normalization_25 (Ba  (None, 112, 112, 64)     

In [ ]:
history = model.fit(X_train,y_train, batch_size = 32, epochs=100, validation_data=(X_val, y_val))

Epoch 1/100
123/123 [==============================] - 26s 139ms/step - loss: 11.9528 - accuracy: 0.0722 - recall: 0.0402 - val_loss: 2.9007 - val_accuracy: 0.0521 - val_recall: 0.0000e+00
Epoch 2/100
123/123 [==============================] - 16s 133ms/step - loss: 2.9340 - accuracy: 0.0581 - recall: 0.0010 - val_loss: 2.8611 - val_accuracy: 0.0573 - val_recall: 0.0000e+00
Epoch 3/100
123/123 [==============================] - 17s 135ms/step - loss: 2.8152 - accuracy: 0.0773 - recall: 0.0026 - val_loss: 2.7692 - val_accuracy: 0.0521 - val_recall: 0.0000e+00
Epoch 4/100
123/123 [==============================] - 16s 133ms/step - loss: 2.7597 - accuracy: 0.0883 - recall: 0.0115 - val_loss: 2.7150 - val_accuracy: 0.1042 - val_recall: 0.0156
Epoch 5/100
123/123 [==============================] - 16s 133ms/step - loss: 2.7203 - accuracy: 0.1067 - recall: 0.0192 - val_loss: 2.6808 - val_accuracy: 0.0938 - val_recall: 0.0521
Epoch 6/100
123/123 [==============================] - 16s 133ms/st

In [ ]:
model_path = '/content/drive/MyDrive/Thoa/v4.h5'
model.save(model_path)

In [ ]:
import matplotlib.pyplot as plt
#plotting the Accuracy of test and training sets
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig("/content/drive/MyDrive/Thoa/v4_acc.png", dpi=300)
plt.show()

In [ ]:
#plotting the loss of test and training sets
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig("/content/drive/MyDrive/Thoa/v4_loss.png", dpi=300)
plt.show()

In [ ]:
acc_score = model.evaluate(X_train, y_train) #we are starting to test the model here
print('\n', 'Train accuracy:', acc_score[1])

acc_score = model.evaluate(X_val, y_val) #we are starting to test the model here
print('\n', 'Val accuracy:', acc_score[1])

acc_score = model.evaluate(X_test, y_test) #we are starting to test the model here
print('\n', 'Test accuracy:', acc_score[1])

In [ ]:
predictions = model.predict(X_test)
fig = plt.figure(figsize=(16, 9))
for i, idx in enumerate(np.random.choice(X_test.shape[0], size=16, replace=False)):
    ax = fig.add_subplot(4, 4, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(X_test[idx]))
    pred_idx = np.argmax(predictions[idx])
    true_idx = np.argmax(y_test[idx])
    ax.set_title("{} ({})".format(labels[pred_idx], labels[true_idx]),
                 color=("green" if pred_idx == true_idx else "red"))